# Visual comparisons

This notebook is a simple visualization of the differences between two networks by combining tools available in this repository.

To execute this notebook, you can use the pypsa-earth environment

### 1. Initial setup

General imports

In [ ]:
from google_drive_downloader import GoogleDriveDownloader as gdd
from pathlib import Path
import pypsa
import pandas as pd
import geopandas as gpd
from pypsa.clustering.spatial import get_clustering_from_busmap
import numpy as np

#### Specify what networks to compare

In [ ]:
# use_drive enables to download default data from gdrive
# When use_drive is true and the path of the file is not found,
# but it matches a value from dictionary file_ids,
# then the file is downloaded from gdrive
use_gdrive = True  

path_network_1 = "data/pypsa-eur/networks/elec_s_512.nc"
path_network_2 = "data/pypsa-earth/networks/elec_s_110.nc"
gadm_shape = "data/pypsa-earth/resources/shapes/gadm_shapes.geojson"
country_shape = "data/pypsa-earth/resources/shapes/country_shapes.geojson"

comparison_methodology = {
    "method": "shape",
    "options": {
        "path": "data/pypsa-earth/resources/shapes/country_shapes.geojson"
    }
}  # method option among: ["country_shape", "gadm_shape", ...]
# TODO: expand to include network_1 and network_2; example: create voronoi polygons and compare them or alike,
# or "find_closest" to compare the closest nodes

# file_ids of default gdrive data
file_ids = {
    # PyPSA-Eur
    "data/pypsa-eur/networks/elec_s_1024.nc": "1GQxNVwpU62YVlWiupFUVMG0Ipu_3UUUd",  # 200Mb!!!
    "data/pypsa-eur/networks/elec_s_512.nc": "1HiOyMzZGA75LfNhFnGU_ntnmjO9CPfEo",
    "data/pypsa-eur/networks/elec_s_256.nc": "1JphEeBz3vdVKY0uKnnHSf-f4k9223emg",
    "data/pypsa-eur/networks/elec.nc": "16DHvFbNah9LblbXOjIbZ6H0cHmCaYH_U",  # % 500Mb!!
    "data/pypsa-eur/networks/base.nc": "1JphEeBz3vdVKY0uKnnHSf-f4k9223emg",
    # PyPSA-Earth
    "data/pypsa-earth/networks/elec_s_110.nc": "12muoaSDkROjTD5cAw143jh3FBPXufNf5",
    "data/pypsa-earth/networks/elec.nc": "1jEtsC8ma9YnAegMxOV4tQWbE4msIoGqT",  # 2Gb!!!
    "data/pypsa-earth/networks/base.nc": "157l0eo8UbmF1HSXaRVb_0ErWXvhPhNka",
    "data/pypsa-earth/resources/shapes/gadm_shapes.geojson": "1DsAn53rTK7Wz6rnga2ogXEnGpXiH5yN4",
    "data/pypsa-earth/resources/shapes/country_shapes.geojson": "1-KxaGSdSXyOlqSfkYNvvJMjZavXQ4Sih",
}

# global crs parameters
GEO_CRS = "EPSG:4326"
METRIC_CRS = "EPSG:3857"

#### Initial downloads from gdrive for template

In [ ]:
# utility function for gdrive
def download_grive(file_id, dest_path, showsize=False):
    gdd.download_file_from_google_drive(
        file_id=file_id,
        dest_path=dest_path,
        showsize=showsize,
        unzip=False,
    )

files_to_download = [path_network_1, path_network_2, gadm_shape, country_shape]

for fpath in files_to_download:
    pl_path = Path(fpath)

    # skip if file exists
    if pl_path.is_file():
        print(f"File '{fpath}' found")
        continue

    if use_gdrive:
        if fpath in file_ids:
            pl_path.parent.mkdir(parents=True, exist_ok=True)
            download_grive(file_ids[fpath], fpath)
        else:
            print(f"File '{fpath}' not found and not in file_ids")
            raise FileNotFoundError(fpath)
    else:
        print(f"File '{fpath}' found")
        raise FileNotFoundError(fpath)

### 2. Comparison methodology

In [ ]:
# Load networks
n1 = pypsa.Network(path_network_1)  # first network
n2 = pypsa.Network(path_network_2)  # second network

# Utility function for dataframe to geodataframe conversion
def buses_to_geodf(df_buses, INPUT_CRS=GEO_CRS, OUTPUT_CRS=METRIC_CRS):
    """Function to transform a buses dataframe into a geodataframe with the correct crs."""
    return gpd.GeoDataFrame(
        df_buses,
        geometry=gpd.points_from_xy(df_buses.x, df_buses.y),
        crs=INPUT_CRS,
    ).to_crs(OUTPUT_CRS)


### Create mapping of the networks for comparison purposes

##### Utility functions for mapping

In [ ]:
def shape_mapping(n, options):
    """Create mapping by shape"""
    gdf = gpd.read_file(options["path"])

    # create GADM_ID and country if missing (country_shapes.geojson)
    if "GADM_ID" not in gdf.columns:
        gdf["GADM_ID"] = gdf["name"]
    if "country" not in gdf.columns:
        gdf["country"] = gdf["name"]
    
    gdf.set_index("GADM_ID", inplace=True)
    df_mapped = n.buses.groupby("country").apply(
        lambda x: gpd.sjoin_nearest(buses_to_geodf(x), gdf[gdf.country==x.name].to_crs(METRIC_CRS), how="inner")
    ).droplevel(0, axis=0)
    return (df_mapped["index_right"] + " " + df_mapped.carrier).rename("mapping")


def create_bus_mapping(n, method):
    """Function to create the bus mapping of network n according to a given methodology"""
    match method["method"]:
        case "shape":
            return shape_mapping(n, method["options"])
        case _:
            raise NotImplementedError(f"Method {method['method']} not implemented")

In [ ]:
aggregation_strategies = {
    "generators": {  # use "min" for more conservative assumptions
        "p_nom": "sum",
        "p_nom_max": "sum",
        "p_nom_min": "sum",
        "p_min_pu": "mean",
        "marginal_cost": "mean",
        "committable": "any",
        "ramp_limit_up": "max",
        "ramp_limit_down": "max",
        "efficiency": "mean",
    }
}

def get_aggregation_strategies(aggregation_strategies):
    """
    Default aggregation strategies that cannot be defined in .yaml format must
    be specified within the function, otherwise (when defaults are passed in
    the function's definition) they get lost when custom values are specified
    in the config.
    """
    import numpy as np

    # to handle the new version of PyPSA.
    try:
        from pypsa.clustering.spatial import _make_consense
    except Exception:
        # TODO: remove after new release and update minimum pypsa version
        from pypsa.clustering.spatial import _make_consense

    bus_strategies = dict(country=_make_consense("Bus", "country"))
    bus_strategies.update(aggregation_strategies.get("buses", {}))

    generator_strategies = {"build_year": lambda x: 0, "lifetime": lambda x: np.inf}
    generator_strategies.update(aggregation_strategies.get("generators", {}))

    return bus_strategies, generator_strategies

# Bus aggregation strategies

def create_clustering(n, busmap, aggregation_strategies=aggregation_strategies):
    # get aggregation strategies
    bus_strategies, generator_strategies = get_aggregation_strategies(aggregation_strategies)

    # get clustering
    clustering = get_clustering_from_busmap(
        n,
        busmap,
        bus_strategies=bus_strategies,
        aggregate_generators_weighted=True,
        aggregate_generators_carriers=None,
        aggregate_one_ports=["Load", "StorageUnit"],
        line_length_factor=1.0,
        generator_strategies=generator_strategies,
        scale_link_capital_costs=False,
    )
    return clustering.network, busmap

In [ ]:
def create_clustered_network(n, method):
    return create_clustering(n, create_bus_mapping(n, method))

#### Execute the mapping

In [ ]:
n1_mapped, n1_mapped_busmap = create_clustered_network(n1, comparison_methodology)
n2_mapped, n2_mapped_busmap = create_clustered_network(n2, comparison_methodology)


### 3. Execute the comparison

#### General statistics by networks

In [ ]:
s1 = n1_mapped.statistics()
s1

In [ ]:
s2 = n2_mapped.statistics()
s2

Calculate percentage difference [\%]

In [ ]:
delta_mapped = s2 - s1
delta_mapped_pc = delta_mapped / s1 * 100
delta_mapped_pc

In [ ]:
n2.plot()

#### Compare lines

In [ ]:
# utility function
def add_line_names(lines):
    """Prepare line names for comparison"""
    lines["carrier"] = lines.bus0.str.split().str[1]
    lines["name"] = lines.bus0.str.split().str[0] + " - " + lines.bus1.str.split().str[0]

    return lines

Compare lines as dataframe

In [ ]:
lines1 = add_line_names(n1_mapped.lines.copy()).set_index("name")
lines2 = add_line_names(n2_mapped.lines.copy()).set_index("name")

carrier = pd.concat([lines1.carrier, lines2.carrier[lines2.index.difference(lines1.index)]], ignore_index=False)

delta_lines = pd.concat([carrier, lines1.s_nom.rename("lines 1"), lines2.s_nom.rename("lines 2")], axis=1).fillna(0)
delta_lines["bus0"] = delta_lines.index.str.split(" - ").str[0] 
delta_lines["bus1"] = delta_lines.index.str.split(" - ").str[1] 
delta_lines["delta"] = delta_lines["lines 2"] - delta_lines["lines 1"]
delta_lines["delta_pu"] = (delta_lines["delta"] / (delta_lines["lines 1"] + delta_lines["lines 2"]) * 2).fillna(0)
delta_lines["delta_pc"] = delta_lines["delta_pu"] * 100

header_cols = ["carrier", "bus0", "bus1", "lines 1", "lines 2", "delta", "delta_pu", "delta_pc"]
delta_lines = delta_lines[header_cols]
delta_lines